<center> <h1> Stokes configuration : Probleme_FT_Disc_gen </h1></center>


In [ ]:
from trustutils import run
run.introduction("Edouard Butaye")
run.TRUST_parameters()

## Problem description

This form allows comparing the hydrodynamic forces computed during the simulation with the theoretical forces under the Stokes flow assumption. The particle settles in a viscous fluid under the effect of gravity. The domain is a cube with a side length of 3 particle diameters. The NS equations are solved in the reference frame of the particle. To do this, we impose the fluid velocity at the domain boundaries. This can be Computed by analytically solving the Stokes flow.


                                       imposed pressure and velocity
                                     ________________________________
                                    |                                |
                                    |                                |
                                    |          /-----------\         | 
                    imposed velociy |         /             \        | imposed velocity
                                    |        |    sperical   |       |
                                    |        |    particle   |    |  |
                                    |         \   dimeter D /   g |  |
                                    |          \-----------/      |  |
                                    |                            \ / |
                                    |                                |
                                    |________________________________|

                                           imposed velocity


## Computation set up
### Geometric parameters
- Origin : -7.8e-5 -7.8e-5 -7.8e-5
- Node numbers : 16 16 16
- Lengths : 15.6e-5 15.6e-5 15.6e-5
### Physical parameters
- Particle :
      - mu  3 Pa.s
      - rho 10000 kg.m^{-3}
      - e_dry 0.97
      - radius  2.6e-5 m
- Fluid :
      - mu  3e-3 Pa.s
      - rho 1e3 
- Gravity : -10 m.s^{-2}
- Surface tension : 0 N/m (not considered for a solid particle) 
### Numerical parameters
- Time scheme : Euler explicite + diffusion implicite
- Threshold implicite diffusion : 1e-11
- Time step : 10^{-6} s
- Total computation time : 10^{-3} s
- No remeshing (non-deformable particle)
- Velocity interpolation : VDF_lineaire

In [ ]:
run.reset()
run.initBuildDirectory()

dic_projected_tensor = {
    "method_friction_force_computation": "trilinear_linear_projected_tensor",
    "formule_mu": "harmonic"
}
dic_complete_tensor = {
    "method_friction_force_computation": "trilinear_linear_complete_tensor",
    "formule_mu": "harmonic"
}

type_interp = {
    "projected_tensor": dic_projected_tensor,
    "complete_tensor": dic_complete_tensor
}

dic_arithmetic = {
    "method_friction_force_computation": "trilinear_linear_projected_tensor",
    "formule_mu": "arithmetic"
}
dic_harmonic = {
    "method_friction_force_computation": "trilinear_linear_projected_tensor",
    "formule_mu": "harmonic"
}

type_visc = {
    "arithmetic": dic_arithmetic,
    "harmonic": dic_harmonic
}

for visc, vvisc in type_visc.items():
        target_repo = f"{visc}"
        run.addCaseFromTemplate("stokes.data", target_repo, {**vvisc})


for interp, vinterp in type_interp.items():
        target_repo = f"{interp}"
        run.addCaseFromTemplate("stokes.data", target_repo, {**vinterp})


run.printCases()

In [ ]:
run.runCases()

### Performances

In [ ]:
table = run.tablePerf()
table = table.drop(columns=["host", "system"]).drop("Total")
table

## Theoretical fields
### Computation of the terminal velocity.

\begin{equation}
U_{inf}=\frac{2}{3}\frac{gr_p^2}{\mu_f}\frac{1+\Phi_\mu}{2+3\Phi_\mu}\left(\rho_f-\rho_p\right)
\end{equation}

In [ ]:
Dp=5.2e-5
rp=Dp/2
g=-10
rho_p=10000
rho_f=1000
xp=0
mu_p=3e-0
mu_f=3e-3
phi_mu=mu_p/mu_f #phi_mu représente la rapport de viscosité entre phases
Uinf=2/3*g*rp**2/mu_f*(1+phi_mu)/(2+3*phi_mu)*(rho_p-rho_f)

print(f"The terminal setting velocity is U_inf = {Uinf:.2e} m/s.")

### Computation of the velocity field


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from math import pi,sqrt

cm_to_inches=0.393701
width=10.5*cm_to_inches
height=7.5*cm_to_inches
res_plot_part=100
res_plot=500
PHI_MU_EQ=True

def init_fig(diameter, Nd_width,Nd_height,title):
    fig=plt.figure(figsize=[width,height])
    ax=fig.add_subplot()
    plt.axis('equal')
    ry=(Nd_width*diameter/2)/diameter 
    rz=(Nd_height*diameter/2)/diameter 
    ax.set_xlabel(r'$y/D_p$')
    ax.set_ylabel(r'$z/D_p$')
    ax.set_xlim(-ry+1,ry-1)
    ax.set_ylim(-rz,rz)
    ax.set_title(title)
    return(fig,ax,ry,rz)

def plot_particle(diameter):
    theta=np.linspace(0,2*pi,res_plot_part)
    radius=diameter/2
    x1=radius*np.cos(theta)/diameter
    x2=(radius*np.sin(theta))/diameter
    plt.plot(x1,x2, linewidth=1,c='black')
    
def consider_phi_mu(check_phi_mu):
    if check_phi_mu:
        return ((2+3*phi_mu)/(1+phi_mu),phi_mu/(1+phi_mu))
    else:
        return (3,1)
fac_r,fac_r3=consider_phi_mu(PHI_MU_EQ)

def vel_x(x,y,z,radius,Uinf): # rappel : On suppose ici que la particule est centré en x=z=0 et tombe par gravité suivant y 
    if sqrt(x**2+y**2+z**2)<radius:
        velx = Uinf/(2*radius**2)*(1/(1+phi_mu))*x*z
    else:
        velx = Uinf*z*x*(fac_r*(radius)/(4*((x**2+y**2+z**2)**1.5))-3*fac_r3*(radius)**3/(4*(x**2+y**2+z**2)**2.5))
    return(velx)

def vel_y(x,y,z,radius,Uinf): # rappel : On suppose ici que la particule est centré en x=z=0 et tombe par gravité suivant y 
    if sqrt(x**2+y**2+z**2)<radius:
        vely = Uinf/(2*radius**2)*(1/(1+phi_mu))*y*z
    else:
        vely = Uinf*z*y*(fac_r*(radius)/(4*((x**2+y**2+z**2)**1.5))-3*fac_r3*(radius)**3/(4*(x**2+y**2+z**2)**2.5))
    return(vely)

def vel_z(x,y,z,radius,Uinf): # rappel : On suppose ici que la particule est centré en x=z=0 et tombe par gravité suivant y 
    if sqrt(x**2+y**2+z**2)<radius:
        velz =  Uinf/(2*(1+phi_mu))*(1+z**2/(radius**2)-2*(x**2+y**2+z**2)/(radius**2))
    else:
        velz = -Uinf*(+(z**2)*(1/(x**2+y**2+z**2)-fac_r*(radius)/(2*((x**2+y**2+z**2)**1.5))+fac_r3*radius**3/(2*(x**2+y**2+z**2)**2.5))+
                      (1-z**2/(x**2+y**2+z**2))*(1-fac_r*radius/(4*(x**2+y**2+z**2)**0.5)-fac_r3*radius**3/(4*(x**2+y**2+z**2)**1.5)))

    return(velz)

def set_velocity_field(x0, Y, Z, radius, Uinf):
    velocity_field_z=np.zeros((len(Y[0]),len(Y[0])))
    velocity_field_y=np.zeros((len(Y[0]),len(Y[0])))

    for i in range(len(Y)):
        for j in range(len(Y[i])):
            velocity_field_z[i,j]=vel_z(x0, Y[i,j]*Dp, Z[i,j]*Dp, radius*Dp, Uinf)
            velocity_field_y[i,j]=vel_y(x0, Y[i,j]*Dp, Z[i,j]*Dp, radius*Dp, Uinf)
    return(velocity_field_y, velocity_field_z)
    
def init_velocity_field(ry,rz,radius, Uinf):
    CoordY=np.linspace(-ry,ry,res_plot)
    CoordZ=np.linspace(-rz,+rz,res_plot)
    extent = np.min(CoordY), np.max(CoordY), np.min(CoordZ), np.max(CoordZ)
    Y,Z=np.meshgrid(CoordY,CoordZ)
    Z=-Z
    velocity_field_y, velocity_field_z = set_velocity_field(xp, Y, Z, radius, Uinf)
    return(Y, Z, velocity_field_y,velocity_field_z, extent)


Nd_width,Nd_height=3.5,2.5
fig,ax,ry,rz=init_fig(Dp, Nd_width, Nd_height, r'Z-component of th velocity field in the plane $x=0$')
Y,Z,velocity_field_y,velocity_field_z,extent=init_velocity_field(ry, rz, rp/Dp, Uinf)
im=plt.imshow(velocity_field_z, cmap=plt.cm.jet,extent=extent)
cbar=plt.colorbar(im)
plot_particle(Dp)

fig,ax,ry,rz=init_fig(Dp, Nd_width, Nd_height, r'X/Y-component of the velocity field in the plane $x=0$')
im=plt.imshow(velocity_field_y, cmap=plt.cm.jet,extent=extent)
cbar=plt.colorbar(im)
plot_particle(Dp)

### Computation of the pressure field

In [ ]:
plane_z=3*rp
plane_x=0

def init_fig(diameter, Nd_width, Nd_height, title, plane):
    fig=plt.figure(figsize=[width,height])
    ax=fig.add_subplot()
    plt.axis('equal')
    rx=(Nd_width*diameter/2)/(diameter/2)
    ry=(Nd_height*diameter/2)/(diameter/2)
    if plane=='z':
        ax.set_xlabel(r'$x/r_p$')
        ax.set_ylabel(r'$y/r_p$')
    if plane=='x':
        ax.set_xlabel(r'$y/r_p$')
        ax.set_ylabel(r'$z/r_p$')
    ax.set_xlim(-rx+1,rx-1)
    ax.set_ylim(-ry,+ry)
    ax.set_title(title)
    return(fig,ax,rx,ry)
    
def compute_pressure(x, y, radius, Uinf, plane, h_hydro,max_p_int,max_p_ext): # rappel : On suppose ici que la particule est centré en x=z=0 et tombe par gravité suivant y 
    if plane=='z':
        if sqrt(x**2+y**2+plane_z**2)<radius:
            pressure = -mu_p*Uinf*5/(1+phi_mu)*plane_z/(radius**2)  #- rho_f*g*plane_z
            if abs(pressure)>max_p_int:
                max_p_int=abs(pressure)
        else:
            pressure = ((2+3*phi_mu)/(1+phi_mu))*1/2*mu_f*Uinf*radius*plane_z/((x**2+y**2+(plane_z)**2)**1.5) #- rho_f*g*plane_z
            if abs(pressure)>max_p_ext:
                max_p_ext=abs(pressure)
    if plane=='x': # ici x joue le rôle de y et y joue le rôle de z
        if sqrt(plane_x**2+x**2+y**2)<radius:
            pressure = 0#mu_p*Uinf*5/(1+phi_mu)*y/(radius**2)  #- rho_f*g*y
            if abs(pressure)>max_p_int:
                max_p_int=abs(pressure)
        else:
            pressure = mu_f*Uinf*((2+3*phi_mu)/(1+phi_mu))*(radius/2)*y/((plane_x**2+x**2+y**2)**1.5)  #- rho_f*g*y
            if abs(pressure)>max_p_ext:
                max_p_ext=abs(pressure)
    
    return(pressure, max_p_int, max_p_ext)

def set_pressure_field(X, Y, radius, Uinf, plane, h_hydro,max_p_int,max_p_ext):
    pressure_field=np.zeros((len(X[0]),len(X[0])))
   
    for i in range(len(Y)):
        for j in range(len(Y[i])):
            pressure_field[i,j], max_p_int, max_p_ext =compute_pressure(X[i,j]*rp, Y[i,j]*rp, radius*rp, Uinf, plane, h_hydro,max_p_int,max_p_ext)

    return(pressure_field)
    
def init_pressure_field(rx, ry, radius, Uinf, plane, h_hydro):
    max_p_int,max_p_ext=0,0
    CoordX=np.linspace(-rx,rx,res_plot)
    CoordY=np.linspace(-ry,+ry,res_plot)
    extent = np.min(CoordX), np.max(CoordX), np.min(CoordY), np.max(CoordY)
    X,Y=np.meshgrid(CoordX, CoordY)
    Y=-Y
    pressure_field=set_pressure_field(X, Y, radius, Uinf, plane, h_hydro,max_p_int,max_p_ext)
    return(X, Y, pressure_field, extent)


#%% Tracé des figures
# Figure dans le plane z
plane='z'
fig,ax,rx,ry=init_fig(Dp, Nd_width, Nd_height, r'Pressure field in the plane $z=1.5 Dp$', plane)
X,Y,pressure_field,extent=init_pressure_field(rx, ry, rp/Dp, Uinf, plane, Nd_height*Dp)

im=plt.imshow(pressure_field, cmap=plt.cm.jet,extent=extent)
cbar=plt.colorbar(im)
cbar.set_label(r'$P_k\,/\,|max(P_k)|$')
plot_particle(Dp)
#%% Figure dans le plane x
plane='x'
fig,ax,rx,ry=init_fig(Dp, Nd_width, Nd_height, r'Pressure field in the plane $x=0$',plane)
X,Y,pressure_field,extent=init_pressure_field(rx, ry, rp/Dp, Uinf, plane,Nd_height*Dp)

im=plt.imshow(pressure_field, cmap=plt.cm.jet,extent=extent)
cbar=plt.colorbar(im)
cbar.set_label(r'$P\,/P_{atm}$')
plot_particle(Dp)

### Computation of hydrodynamic theoretical forces

Friction force
\begin{equation}
F_f = -4\pi \mu_f r_p U_{inf}
\end{equation}
Pressure force
\begin{equation}
F_f = -2\pi \mu_f r_p U_{inf}
\end{equation}
Total hydrodynamic force
\begin{equation}
F_f = -6\pi \mu_f r_p U_{inf}
\end{equation}


In [ ]:
Ff_th=-4*pi*mu_f*rp*Uinf
Fp_th=-2*pi*mu_f*rp*Uinf
Fh_th=-6*pi*mu_f*rp*Uinf
print(f"Friction force : {Ff_th:.2e} N")
print(f"Pressure force : {Fp_th:.2e} N")
print(f"Total hydrodynamic force : {Fh_th:.2e} N")

## Results
### Eulerian fluid & particle velocity

The following plot shows a slice of the z-component (vertical direction) of the eulerian fluid velocity at the center of the domain.

In [ ]:
from trustutils import visit
fig=visit.Show(f"./{interp}/lata/post_dom.lata", "Pseudocolor", "VITESSE_Z_ELEM_dom")
fig.slice(var="y",all=1,type_op="slice2d")
fig.plot()

The following plot shows a slice of the X and y -components (horizontal directions) of the eulerian fluid velocity at the center of the domain.

In [ ]:
fig=visit.Show(f"./{interp}/lata/post_dom.lata", "Pseudocolor", "VITESSE_X_ELEM_dom")
fig.slice(var="y",all=1,type_op="slice2d")
fig.plot()

In [ ]:
fig=visit.Show(f"./{interp}/lata/post_dom.lata", "Pseudocolor", "VITESSE_Y_ELEM_dom")
fig.slice(var="x",all=1,type_op="slice2d")
fig.plot()

The following plots show the pressure field at the center of the domain. Be aware, the pressure field inside the particle is several order of magnitude larger than the pressure field of the fluid. Consequently, visualization of the pressure field without any post-processing will give no information outside the particle.

In [ ]:
fig=visit.Show(f"./{interp}/lata/post_dom.lata", "Pseudocolor", "PRESSION_ELEM_dom")
fig.slice(var="z",all=1,type_op="slice2d")
fig.plot()

In [ ]:
fig=visit.Show(f"./{interp}/lata/post_dom.lata", "Pseudocolor", "PRESSION_ELEM_dom")
fig.slice(var="y",all=1,type_op="slice2d")
fig.plot()

### Lagrangian particle velocity

In the present study, NS equations are solved in the reference frame of the particle. Therefore, if the flow is well resolved, the particle's velocity is zero. If not, this is due to a lack of resolution of hydrodynamic forces. For a resolution of 10 meshes per particle diameter, the 

In [ ]:
from trustutils import plot
import itertools
import matplotlib.pyplot as plt
import numpy as np
marker = itertools.cycle(('^', '+', 'd', 'o', '*', '<', '>', 's'))

dcolors = {"projected_tensor": "blue", "complete_tensor": "red"}
dimension=3

In [ ]:
ylabel=["Ux","Uy", "Uz"]

for dim in range(dimension):
    fig,ax=plt.subplots()
    ax.set_ylabel(f"{ylabel[dim]} [m/s]", fontsize=15)
    ax.set_xlabel(r"$time [s]$", fontsize=15)
    
    for interp in type_interp.keys():
        time,vel=np.loadtxt(f"{run.BUILD_DIRECTORY}/{interp}/stokes_particles_trajectory_interf.out",skiprows=10,usecols=(0, 4+dim),unpack=True)
        ax.plot(time,vel,label=f"{interp}",lw=1,color=dcolors[interp])
        ax.legend(loc="best")


### Hydrodynamic forces

To assess deviations to theoretical forces, the reader is referred to figures 5 and 10 of Butaye et al, 2023 : https://www.sciencedirect.com/science/article/pii/S0045793023002967?via%3Dihub 

In [ ]:
ylabel=["Fp","Ff", "Fh"]

for dim in range(dimension):
    fig,ax=plt.subplots()
    ax.set_ylabel(f"{ylabel[dim]} [N]", fontsize=15)
    ax.set_xlabel(r"$time [s]$", fontsize=15)
    ind=0
    for interp in type_interp.keys():
        if (dim==0):
            time,F=np.loadtxt(f"{run.BUILD_DIRECTORY}/{interp}/stokes_particle_hydrodynamic_forces_interf.out",skiprows=10,usecols=(0, 4),unpack=True)
            if (ind==0): ax.axhline(Fp_th,c='black', label="Theoretical value")
        elif (dim==1):
            time,F=np.loadtxt(f"{run.BUILD_DIRECTORY}/{interp}/stokes_particle_hydrodynamic_forces_interf.out",skiprows=10,usecols=(0, 7),unpack=True)
            if (ind==0): ax.axhline(Ff_th,c='black', label="Theoretical value")
        else:
            time,Fp=np.loadtxt(f"{run.BUILD_DIRECTORY}/{interp}/stokes_particle_hydrodynamic_forces_interf.out",skiprows=10,usecols=(0, 4),unpack=True)
            _,Ff=np.loadtxt(f"{run.BUILD_DIRECTORY}/{interp}/stokes_particle_hydrodynamic_forces_interf.out",skiprows=10,usecols=(0, 7),unpack=True)
            F=Fp+Ff
            if (ind==0): ax.axhline(Fp_th+Ff_th,c='black', label="Theoretical value")
        ax.plot(time,F,label=f"{interp}",lw=1,color=dcolors[interp])
        ax.legend(loc='best')
        ind+=1


# Computation of the viscosity

It exists several methods to compute the viscosity in two-phase cells as described below.  
I represent the volumic phase indicator function for the fluid. $I==1$ indicates a fully fluid cell. $I==0$ indicates a fully solid cell

* arithmetic average \begin{equation} \mu = I\mu_f + (1-I)\mu_p \end{equation}
* harmonic average \begin{equation} \mu = \frac{\mu_p\mu_f}{(1-I)\mu_f+I\mu_p} \end{equation}
* staircase average 
$$\mu= \mu_f\: \text{if}\: I>0;\: \mu_p\: \text{otherwise}$$

The computation of the phase indicator function remain challenging in staggered control volumes. One method is to apply the phase indicator function calculation method to staggered control volumes.  Another option would be to compute the average value of the volumic phase indicator function in neighboring cells.  

In TrioCFD-1.9.6, the phase indicator function is computed accurately in cells and averaged in staggered cells. Therefore, off-diagonal components of the stress tensor are computed with a viscosity interpolated at the gravity center of the edges. The staircase average is not present in TrioCFD-1.9.6.


In [ ]:
ylabel=["Ux","Uy", "Uz"]
dcolors = {"arithmetic": "blue", "harmonic": "red"}
for dim in range(dimension):
    fig,ax=plt.subplots()
    ax.set_ylabel(f"{ylabel[dim]} [m/s]", fontsize=15)
    ax.set_xlabel(r"$time [s]$", fontsize=15)
    
    for visc, vvisc in type_visc.items():
              time,vel=np.loadtxt(f"{run.BUILD_DIRECTORY}/{visc}/stokes_particles_trajectory_interf.out",skiprows=10,usecols=(0, 4+dim),unpack=True)
              ax.plot(time,vel,label=f"{visc}",lw=1,color=dcolors[visc])
              ax.legend(loc="best")